## Imports

In [34]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from IPython.display import Image
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from shutil import copy2, rmtree
from PIL import Image
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

## Lendo o Dataset

In [35]:
pokemon_df = pd.read_csv('pokemon.csv').drop(columns=['Type2'])
pokemon_df

,Name,Type1
0,bulbasaur,Grass
1,ivysaur,Grass
2,venusaur,Grass
3,charmander,Fire
4,charmeleon,Fire
...,...,...
804,stakataka,Rock
805,blacephalon,Fire
806,zeraora,Electric
807,meltan,Steel


### Adicionando path da imagem na row correspondente

Algumas imagens possuem fundo transparente, então foi feita a conversão para RGBA

In [36]:
pokemon_imgs = os.listdir('images')

if not os.path.exists('converted_images/'):
    os.mkdir('converted_images/')
else:
    rmtree('converted_images/')
    os.mkdir('converted_images/') 

sorted_imgs = pokemon_imgs.copy()
for pokemon in pokemon_imgs:
    pk_img = Image.open(f'images/{pokemon}')
    
    pk_img = pk_img.convert("RGBA")
    background = background = Image.new('RGB', pk_img.size, (256,256,256))
    background.paste(pk_img, mask=pk_img.split()[3])
    
    background.save(f'converted_images/{pokemon.split(".")[0]}.jpg')
    
    pokemon_name = pokemon.split('.')[0]
    index = pokemon_df[pokemon_df.Name == pokemon_name].index.values[0]
    sorted_imgs[index] = f'{pokemon.split(".")[0]}.jpg'

In [37]:
pokemon_df['Pokemon Image'] = sorted_imgs
pokemon_df

,Name,Type1,Pokemon Image
0,bulbasaur,Grass,bulbasaur.jpg
1,ivysaur,Grass,ivysaur.jpg
2,venusaur,Grass,venusaur.jpg
3,charmander,Fire,charmander.jpg
4,charmeleon,Fire,charmeleon.jpg
...,...,...,...
804,stakataka,Rock,stakataka.jpg
805,blacephalon,Fire,blacephalon.jpg
806,zeraora,Electric,zeraora.jpg
807,meltan,Steel,meltan.jpg


In [38]:
pokemon_df['Type1'].value_counts()

Water       114
Normal      105
Grass        78
Bug          72
Fire         53
Psychic      53
Rock         46
Electric     40
Poison       34
Ground       32
Fighting     29
Dark         29
Dragon       27
Ghost        27
Steel        26
Ice          23
Fairy        18
Flying        3
Name: Type1, dtype: int64

In [39]:
index = pokemon_df.loc[(pokemon_df.Type1 != 'Water') &  (pokemon_df.Type1 != 'Normal')].index.values
pokemon_df.drop(index=index, inplace=True)

### Separando imagens em Treino e Validação

Add Teste

In [40]:
if not os.path.exists('train/'):
    os.mkdir('train/')
else:
    rmtree('train/')
    os.mkdir('train/') 

if not os.path.exists('val/'):
    os.mkdir('val/')
else:
    rmtree('val/')
    os.mkdir('val/') 

if not os.path.exists('test/'):
    os.mkdir('test/')
else:
    rmtree('test/')
    os.mkdir('test/') 

In [41]:
for pk_type in pokemon_df.Type1.unique():
    if not os.path.exists(f'train/{pk_type}/'):
        os.mkdir('train/'+str(pk_type)+'/')
    if not os.path.exists(f'val/{pk_type}/'):
        os.mkdir('val/'+str(pk_type)+'/')
    if not os.path.exists(f'test/'):
        os.mkdir('test/')

In [42]:
X_train_val, X_test, y_train_val, y_test = train_test_split(pokemon_df['Pokemon Image'], pokemon_df.Type1, 
                                                    test_size=0.2, stratify=pokemon_df.Type1, random_state=42)

In [43]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, 
                                                    test_size=0.2, stratify=y_train_val, random_state=42)

In [44]:
for from_path, to_path in zip(X_train.values, y_train.values):
    if os.path.exists( f'train/{to_path}'):
        copy2(f'converted_images/{from_path}', f'train/{to_path}/')
    

In [45]:
for from_path, to_path in zip(X_test.values, y_test.values):
    copy2(f'converted_images/{from_path}', f'test/')

In [46]:
for from_path, to_path in zip(X_val.values, y_val.values):
    if os.path.exists( f'val/{to_path}'):
        copy2(f'converted_images/{from_path}', f'val/{to_path}/')

In [63]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=40, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=.2, 
                                   zoom_range=0.2,
                                   horizontal_flip=True) 
                                   #fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory('train/', 
                                                    target_size = (64, 64), 
                                                    class_mode='binary', 
                                                    shuffle=True,
                                                    color_mode='rgb')

val_generator = val_datagen.flow_from_directory('val/', 
                                                  target_size = (64, 64),
                                                  class_mode='binary',
                                                  shuffle=True)

Found 140 images belonging to 2 classes.
Found 35 images belonging to 2 classes.


## Testando CNNs

In [65]:
# Initialising the CNN
classifier = Sequential()
# Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
# Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Flattening
classifier.add(Flatten())
# Full connection
#classifier.add(Dense(units = 256, activation = 'relu'))
# Add Dropout to prevent overfitting
#classifier.add(Dropout(0.5))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [49]:
# train_data_augment = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
# test_data_augment = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# batch_size = 32
# train_data = train_data_augment.flow_from_directory('train/', target_size=(112, 112), batch_size=batch_size, 
#                                                     class_mode='categorical', shuffle=True, color_mode='rgb')

# test_data = test_data_augment.flow_from_directory('test/', target_size=(112, 112), batch_size=batch_size, 
#                                                   class_mode='categorical', shuffle=True, color_mode='rgb')
                                                  
# data_augmentation = keras.Sequential([
#     layers.experimental.preprocessing.RandomFlip("horizontal"),
#     layers.experimental.preprocessing.RandomRotation(0.2)
# ])

# augmented_train = train_datagen.map(lambda x, y: (data_augmentation(x, training=True), y))

# augmented_train = augmented_train.prefetch(buffer_size=32)
# validation = val_datagen.prefetch(buffer_size=32)                                                  

In [50]:
# classifier.fit_generator(train_data, steps_per_epoch=len(train_data)/batch_size, epochs=30, 
#                          validation_data=test_data, validation_steps=len(test_data)//batch_size)

In [66]:
classifier.fit(train_generator,
               epochs = 30,
               validation_data = val_generator)

Epoch 1/30
5/5 [==============================] - 1s 178ms/step - loss: 0.8851 - accuracy: 0.5402 - val_loss: 0.7000 - val_accuracy: 0.5143
Epoch 2/30
5/5 [==============================] - 1s 136ms/step - loss: 0.6982 - accuracy: 0.5086 - val_loss: 0.6915 - val_accuracy: 0.6000
Epoch 3/30
5/5 [==============================] - 1s 121ms/step - loss: 0.6903 - accuracy: 0.6249 - val_loss: 0.6902 - val_accuracy: 0.5714
Epoch 4/30
5/5 [==============================] - 1s 122ms/step - loss: 0.6952 - accuracy: 0.4898 - val_loss: 0.6879 - val_accuracy: 0.5429
Epoch 5/30
5/5 [==============================] - 1s 118ms/step - loss: 0.6870 - accuracy: 0.5864 - val_loss: 0.6846 - val_accuracy: 0.5143
Epoch 6/30
5/5 [==============================] - 1s 130ms/step - loss: 0.6786 - accuracy: 0.5511 - val_loss: 0.6755 - val_accuracy: 0.6857
Epoch 7/30
5/5 [==============================] - 1s 118ms/step - loss: 0.6688 - accuracy: 0.6873 - val_loss: 0.6696 - val_accuracy: 0.5429
Epoch 8/30
5/5 [====

In [69]:
from keras.preprocessing import image
test_image = image.load_img('test/snorlax.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = np.round(classifier.predict(test_image))
train_generator.class_indices

if result[0][0] == 1:
  prediction = "Normal"
else:
  prediction = "Water"

In [70]:
im = Image.open('test/snorlax.jpg')
im.show()
prediction

'Normal'